<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/TFTimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_datareader

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from datetime import date
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.contrib.learn import ModeKeys
import tensorflow.contrib.rnn as rnn
from datetime import datetime, date, timedelta





In [0]:

from pandas.tseries.offsets import BDay
TIMESERIES_COL = 'rawdata'
N_OUTPUTS = 5 # in each sequence, 1-14 are features, and 14-20 is label 
SEQ_LEN = 20
DEFAULTS = 0.0
LSTM_SIZE = 5 # number of hidden layers in each of the LSTM cells
N_INPUTS = SEQ_LEN - N_OUTPUTS
BATCH_SIZE = 20
ROOT_DIR = '/home/mmistroni/tf_logs/rnn-run-{}'


def get_prices(startdate, enddate, symbol):
  import pandas_datareader as pdr
  from datetime import date, timedelta
  print ('--Start:{%s}, end:{%s}', startdate, enddate)
  
  stock_data = pdr.get_data_yahoo(symbol, startdate, enddate)[['Close']].pct_change().fillna(0)
  adjClose = np.stack(stock_data['Close'])
  return adjClose
  
def create_training_data2(inputData):
  print ('Len of input dat ais {}', len(inputData))
  return [np.array(inputData[i * SEQ_LEN: (i + 1) * SEQ_LEN]) 
       for i in range(len(inputData) // SEQ_LEN)]

def create_training_data(inputData):
  print ('AdjClose is of shape {}', inputData.shape)
  return inputData.T.reshape(-1,SEQ_LEN)
  
  
def create_time_series(end_date, numDays=280, symbol='XOM'):
  print("==== CREATING TIMESERIES GOING BACK 280 DAYS FROM  {}".format(end_date))
  endDate = end_date # training from 20 days ago
  startDate = endDate - BDay(numDays)
  prices =  get_prices(startDate, endDate, symbol=symbol)
  return create_training_data2(prices)
  


In [0]:

def create_train_and_test(end_date, symbol='XOM'):
  all_timeseries =  create_time_series(end_date, symbol=symbol) #[create_time_series() for i in range(0, SEQ_LEN * 4)]
  all_data = np.stack(all_timeseries)
  print('All data shape is{0} and type {1}'.format(all_data.shape,type(all_data)))
  X, y = all_data[...,0:-N_OUTPUTS], all_data[...,-N_OUTPUTS:]
  print ('X is fo type {0}, y  of type {1}'.format(type(X[0][0]), type(y)))
  print ('X.shape is {0}, y shap is {1}'.format(X.shape, y.shape))
  X_train, X_test, y_train, y_test = train_test_split(X,
                                                      y,
                                                      test_size=0.1,
                                                      random_state=1)
  return X_train, y_train, X_test, y_test



<h3> Creating RNN Model </h3>


In [0]:

# create the inference model
def simple_rnn(features, labels, mode, params):
  # 0. Reformat input shape to become a sequence
  print ('IN Features are:{0}'.format(features))
  x = tf.split(features[TIMESERIES_COL], N_INPUTS, 1)
  #print 'x={}'.format(x)
    
  # 1. configure the RNN
  lstm_cell = rnn.BasicLSTMCell(LSTM_SIZE, forget_bias=1.0)
  outputs, _ = tf.nn.static_rnn(lstm_cell, x, dtype=tf.float32)

  # slice to keep only the last cell of the RNN
  outputs = outputs[-1]
  #print 'last outputs={}'.format(outputs)
  
  # output is result of linear activation of last layer of RNN
  weight = tf.Variable(tf.random_normal([LSTM_SIZE, N_OUTPUTS]))
  bias = tf.Variable(tf.random_normal([N_OUTPUTS]))
  predictions = tf.matmul(outputs, weight) + bias
    
  # 2. loss function, training/eval ops
  if mode == ModeKeys.TRAIN or mode == ModeKeys.EVAL:
     loss = tf.losses.mean_squared_error(labels, predictions)
     train_op = tf.contrib.layers.optimize_loss(
         loss=loss,
         global_step=tf.train.get_global_step(),
         learning_rate=0.01,
         optimizer="SGD") # SGD
     eval_metric_ops = {
      "rmse": tf.metrics.root_mean_squared_error(labels, predictions)
     }
  else:
     loss = None
     train_op = None
     eval_metric_ops = None
  
  # 3. Create predictions
  predictions_dict = {"predicted": predictions}

  # 4. Create export outputs  
  export_outputs = {"predicted": tf.estimator.export.PredictOutput(predictions)}

  # 5. return ModelFnOps
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=predictions_dict,
      loss=loss,
      train_op=train_op,
      eval_metric_ops=eval_metric_ops,
      export_outputs=export_outputs)



<h3> Creating Serving Function , Train Function and Test Function </h3>

In [0]:
def serving_input_receiver_fn():
  feature_placeholders = {
    TIMESERIES_COL: tf.placeholder(tf.float32, [None, N_INPUTS])
  }

  features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
  }

  features[TIMESERIES_COL] = tf.squeeze(features[TIMESERIES_COL], axis=[2], name='timeseries')
  
  print('serving: features={}'.format(features[TIMESERIES_COL]))

  return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)



# Creating a TrainFn and a TestFn
def _train_fn(X, y, batch_size):
    
    def _train():
        """An input function for training"""
        # Convert the inputs to a Dataset.
        # TODO need to be refactored according to https://medium.com/google-cloud/how-to-do-time-series-prediction-using-rnns-and-tensorflow-and-cloud-ml-engine-2ad2eeb189e8
        # this is not good.
        X_32 = tf.cast(X, tf.float32)
        y_32 = tf.cast(y, tf.float32)
        inputs = tf.concat(X_32, axis=1)
        label = tf.concat(y_32, axis=1)
        features, labels = {TIMESERIES_COL: inputs}, label
        
        return features, labels
        dataset = tf.data.Dataset.from_tensor_slices(features, labels)
        # Shuffle, repeat, and batch the examples.
        dataset = dataset.repeat(None).batch(batch_size)
        # This will now return batches of features, label
        return dataset.make_one_shot_iterator().get_next()
    return _train

def _test_fn(X, y, batch_size):
    def _test():
        """An input function for training"""
        # Convert the inputs to a Dataset.
        X_32 = tf.cast(X, tf.float32)
        y_32 = tf.cast(y, tf.float32)
        inputs = tf.concat(X_32, axis=1)
        label = tf.concat(y_32, axis=1)
        
        features, labels = {TIMESERIES_COL: inputs}, label
        
        return features, labels
        
        dataset = tf.data.Dataset.from_tensor_slices(features, labels)
        # Shuffle, repeat, and batch the examples.
        dataset = dataset.repeat(1).batch(batch_size)
        # This will now return batches of features, label
        return dataset.make_one_shot_iterator().get_next()
    return _test  

 


<h3> Creating Experiment Function and running model </h3>

In [0]:
def experiment_fn(output_dir, X, y, x_tst, y_tst):
    # run experiment
    train_spec = tf.estimator.TrainSpec(
          input_fn=_train_fn(X, y, BATCH_SIZE), max_steps=1500)
    exporter = tf.estimator.FinalExporter('timeseries',
    serving_input_receiver_fn)
    eval_spec = tf.estimator.EvalSpec(
            input_fn=_test_fn(x_tst, y_tst, BATCH_SIZE),
            exporters=[exporter])
    
    estimator = tf.estimator.Estimator(model_fn=simple_rnn, model_dir=output_dir)

    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    return estimator


def run_model(end_date, symbol='XOM'):  
  print('Attempting to generate learning model for {}, using end_date={}'.format(symbol, end_date))
  tf.logging.set_verbosity(tf.logging.INFO)    
  output_dir = ROOT_DIR.format(datetime.utcnow().strftime('%Y%m%d%H%M%S'))  
  X_train, y_train, X_test, y_test = create_train_and_test(end_date, symbol=symbol)
  estimator = experiment_fn(output_dir, X_train, y_train, X_test, y_test)
  print('Xtrain is:{} and of shape:{}', type(X_train), X_train.shape)
  return estimator


<h3> SETTING STARTDATE, ENDDATE AND TICKER </h3>

In [0]:
SYMBOL = 'DIS'#'XOM'
END_DATE= date.today()
START_DATE = END_DATE - BDay(N_INPUTS) # This will be used for predicting future prices. end date will be 15 days ago, start_date 280 days earlier


<h3> Now running model </h3>

In [26]:
print('===================== CONFIGURATION ======================')
print('STARTDATE={}'.format(START_DATE))
print('ENDDATE={}'.format(END_DATE))
print('SYMBOL={}'.format(SYMBOL))
print('=== TRAINING MODEL ON PAST 280 DAYS from {}'.format(START_DATE))
estimator = run_model(START_DATE, symbol=SYMBOL)


===================== CONFIGURATION ======================
STARTDATE=2019-06-12 00:00:00
ENDDATE=2019-07-03
SYMBOL=DIS
=== TRAINING MODEL ON PAST 280 DAYS from 2019-06-12 00:00:00
Attempting to generate learning model for DIS, using end_date=2019-06-12 00:00:00
==== CREATING TIMESERIES GOING BACK 280 DAYS FROM  2019-06-12 00:00:00
--Start:{%s}, end:{%s} 2018-05-16 00:00:00 2019-06-12 00:00:00


I0703 19:36:37.356400 140195635861376 estimator.py:1790] Using default config.
I0703 19:36:37.357666 140195635861376 estimator.py:209] Using config: {'_model_dir': '/home/mmistroni/tf_logs/rnn-run-20190703193636', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f818a8def98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker

Len of input dat ais {} 270
All data shape is(13, 20) and type <class 'numpy.ndarray'>
X is fo type <class 'numpy.float64'>, y  of type <class 'numpy.ndarray'>
X.shape is (13, 15), y shap is (13, 5)
IN Features are:{'rawdata': <tf.Tensor 'concat:0' shape=(11, 15) dtype=float32>}


I0703 19:36:38.540808 140195635861376 estimator.py:1147] Done calling model_fn.
I0703 19:36:38.543679 140195635861376 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0703 19:36:38.737385 140195635861376 monitored_session.py:240] Graph was finalized.
I0703 19:36:38.791863 140195635861376 session_manager.py:500] Running local_init_op.
I0703 19:36:38.812908 140195635861376 session_manager.py:502] Done running local_init_op.
I0703 19:36:39.320550 140195635861376 basic_session_run_hooks.py:606] Saving checkpoints for 0 into /home/mmistroni/tf_logs/rnn-run-20190703193636/model.ckpt.
I0703 19:36:39.646717 140195635861376 basic_session_run_hooks.py:262] loss = 0.3158283, step = 1
I0703 19:36:40.014417 140195635861376 basic_session_run_hooks.py:692] global_step/sec: 271.756
I0703 19:36:40.018138 140195635861376 basic_session_run_hooks.py:260] loss = 0.05768039, step = 101 (0.371 sec)
I0703 19:36:40.141987 140195635861376 basic_session_run_hooks.py:692] global_step/sec: 783.97
I0703

IN Features are:{'rawdata': <tf.Tensor 'concat:0' shape=(2, 15) dtype=float32>}


I0703 19:36:42.937024 140195635861376 estimator.py:1147] Done calling model_fn.
I0703 19:36:42.962088 140195635861376 evaluation.py:255] Starting evaluation at 2019-07-03T19:36:42Z
I0703 19:36:43.038809 140195635861376 monitored_session.py:240] Graph was finalized.
I0703 19:36:43.043889 140195635861376 saver.py:1280] Restoring parameters from /home/mmistroni/tf_logs/rnn-run-20190703193636/model.ckpt-1500
I0703 19:36:43.103560 140195635861376 session_manager.py:500] Running local_init_op.
I0703 19:36:43.124576 140195635861376 session_manager.py:502] Done running local_init_op.
I0703 19:36:43.213216 140195635861376 evaluation.py:167] Evaluation [10/100]
I0703 19:36:43.224003 140195635861376 evaluation.py:167] Evaluation [20/100]
I0703 19:36:43.234580 140195635861376 evaluation.py:167] Evaluation [30/100]
I0703 19:36:43.244093 140195635861376 evaluation.py:167] Evaluation [40/100]
I0703 19:36:43.256253 140195635861376 evaluation.py:167] Evaluation [50/100]
I0703 19:36:43.265846 1401956358

serving: features=Tensor("timeseries:0", shape=(?, 15), dtype=float32)
IN Features are:{'rawdata': <tf.Tensor 'timeseries:0' shape=(?, 15) dtype=float32>}


I0703 19:36:43.949539 140195635861376 estimator.py:1147] Done calling model_fn.
I0703 19:36:43.951153 140195635861376 export_utils.py:170] Signatures INCLUDED in export for Classify: None
I0703 19:36:43.955806 140195635861376 export_utils.py:170] Signatures INCLUDED in export for Regress: None
I0703 19:36:43.957349 140195635861376 export_utils.py:170] Signatures INCLUDED in export for Predict: ['predicted', 'serving_default']
I0703 19:36:43.961131 140195635861376 export_utils.py:170] Signatures INCLUDED in export for Train: None
I0703 19:36:43.962477 140195635861376 export_utils.py:170] Signatures INCLUDED in export for Eval: None
I0703 19:36:43.984742 140195635861376 saver.py:1280] Restoring parameters from /home/mmistroni/tf_logs/rnn-run-20190703193636/model.ckpt-1500
I0703 19:36:44.004975 140195635861376 builder_impl.py:661] Assets added to graph.
I0703 19:36:44.005910 140195635861376 builder_impl.py:456] No assets to write.
I0703 19:36:44.057293 140195635861376 builder_impl.py:421]

Xtrain is:{} and of shape:{} <class 'numpy.ndarray'> (11, 15)


<h3> Now, after training,  Let's do some predictions for future </h3>

In [28]:
from datetime import date
from pprint import pprint
# going back to last month
tf.logging.set_verbosity(tf.logging.INFO)    

def _predict_fn(X) :
    def _predict():
        """An input function for training"""
        # Convert the inputs to a Dataset.
        X_32 = tf.cast(X, tf.float32)
        inputs = tf.concat(X_32, axis=1)
        features = {TIMESERIES_COL: inputs}
        return features
    return _predict

  
#TODO amend this function, the one above should be reused....
def get_prices2(startdate=None, enddate=None, symbol='XOM'):
  import pandas_datareader as pdr
  from datetime import date, timedelta
  print ('--Start:{%s}, end:{%s}', startdate, enddate)
  
  if not startdate and not enddate:
    last_check = date.today() - BDay(SEQ_LEN * 2) 
    enddate = last_check
    startdate = last_check - BDay(122)#timedelta(days=120)
  return pdr.get_data_yahoo(symbol, startdate, enddate)[['Close']]


print('==== GETTING MOST RECENT PRICES======')
stock_data = get_prices2(START_DATE- BDay(1), END_DATE, symbol=SYMBOL)
percentage_changes = stock_data.pct_change().fillna(0)
#First Price
first_price = stock_data.values[0]
print('==============First ever price of the series of shape {} is {}'.format(stock_data.shape, first_price ))
print(stock_data.head(30))
print('=========== PERCENTAGE CHANGES ===========')
print(percentage_changes.head(30))



prices = np.stack(percentage_changes['Close'])
#prices = np.stack(stock_data['Close'])
print('===== PRICES ARE===')

pprint(prices)





==== GETTING MOST RECENT PRICES======
--Start:{%s}, end:{%s} 2019-06-11 00:00:00 2019-07-03
==============First ever price of the series of shape (17, 1) is [135.08000183]
                 Close
Date                  
2019-06-11  135.080002
2019-06-12  135.720001
2019-06-13  141.740005
2019-06-14  141.649994
2019-06-17  140.970001
2019-06-18  139.240005
2019-06-19  140.919998
2019-06-20  142.020004
2019-06-21  140.229996
2019-06-24  139.220001
2019-06-25  139.940002
2019-06-26  140.399994
2019-06-27  139.300003
2019-06-28  139.639999
2019-07-01  141.649994
2019-07-02  142.529999
2019-07-03  142.979996
=========== PERCENTAGE CHANGES ===========
               Close
Date                
2019-06-11  0.000000
2019-06-12  0.004738
2019-06-13  0.044356
2019-06-14 -0.000635
2019-06-17 -0.004801
2019-06-18 -0.012272
2019-06-19  0.012065
2019-06-20  0.007806
2019-06-21 -0.012604
2019-06-24 -0.007202
2019-06-25  0.005172
2019-06-26  0.003287
2019-06-27 -0.007835
2019-06-28  0.002441
2019-07-01  

<h3> Now recalculating price percentages to predict </h3>

In [29]:
# Reshaping data
print ('Prices is of shape:{}', prices.shape)
prices = prices.reshape(-1,N_INPUTS) if prices.shape[0] % 15 == 0 else prices[-15:].reshape(-1,N_INPUTS)
print ('Reshaped Prices is of shape:{}', prices.shape)
print ('-------- PREDICTING -------')
print (type(prices))


acc = []
acc += prices[0].tolist()


for i in range(0,10):
  # Given last 15 days, predict the next 5
  candidate = np.array(acc[-15:]).reshape(-1,15)  # this is the Train #
  pred = estimator.predict(input_fn = _predict_fn(candidate))
  item = next(pred)
  vals = item['predicted'] # this is the predicted
  # Next 5 predicted. Now,  we need to retrain the model to add the 5 predicted
  # Then, as test data, pick the previous 15 and then predict the next one
  
  """
  ATTEMPT TO RETRAIN THE MODEL..
    print("====== Now adding prediction to current train....")
    print("Xtrain of type:{}, candidate of type:{}".format(type(X_train), type(candidate)))
    print("SHapes.Train:{}, Candidate:{}".format(X_train.shape, candidate.shape))
    #X_train = np.vstack([X_train , candidate])
    #y_train = np.vstack([y_train , vals])
    # So Now we have to 
    # 1. retrain the model
    # 2. get the most recent 15 days to 
  
  """
  print ('Round {} Prediction: on {}={}'.format(i, candidate, vals))
  print ('{}={}'.format(type(vals), vals.tolist()))
  acc += vals.tolist()


W0703 19:37:18.765938 140195635861376 estimator.py:1000] Input graph does not use tf.data.Dataset or contain a QueueRunner. That means predict yields forever. This is probably a mistake.
I0703 19:37:18.766930 140195635861376 estimator.py:1145] Calling model_fn.


Prices is of shape:{} (17,)
Reshaped Prices is of shape:{} (1, 15)
-------- PREDICTING -------
<class 'numpy.ndarray'>
IN Features are:{'rawdata': <tf.Tensor 'concat:0' shape=(1, 15) dtype=float32>}


I0703 19:37:18.991616 140195635861376 estimator.py:1147] Done calling model_fn.
I0703 19:37:19.063185 140195635861376 monitored_session.py:240] Graph was finalized.
I0703 19:37:19.068585 140195635861376 saver.py:1280] Restoring parameters from /home/mmistroni/tf_logs/rnn-run-20190703193636/model.ckpt-1500
I0703 19:37:19.101164 140195635861376 session_manager.py:500] Running local_init_op.
I0703 19:37:19.107667 140195635861376 session_manager.py:502] Done running local_init_op.
W0703 19:37:19.174830 140195635861376 estimator.py:1000] Input graph does not use tf.data.Dataset or contain a QueueRunner. That means predict yields forever. This is probably a mistake.
I0703 19:37:19.175960 140195635861376 estimator.py:1145] Calling model_fn.


Round 0 Prediction: on [[ 0.04435606 -0.00063505 -0.00480051 -0.01227208  0.01206545  0.00780589
  -0.01260392 -0.00720241  0.00517168  0.00328706 -0.00783469  0.00244075
   0.01439412  0.00621253  0.00315721]]=[ 6.2403977e-03  4.3793470e-03  5.2854419e-03 -1.8239021e-05
 -6.8126451e-03]
<class 'numpy.ndarray'>=[0.006240397691726685, 0.004379346966743469, 0.005285441875457764, -1.823902130126953e-05, -0.006812645122408867]
IN Features are:{'rawdata': <tf.Tensor 'concat:0' shape=(1, 15) dtype=float32>}


I0703 19:37:19.419659 140195635861376 estimator.py:1147] Done calling model_fn.
I0703 19:37:19.491446 140195635861376 monitored_session.py:240] Graph was finalized.
I0703 19:37:19.496734 140195635861376 saver.py:1280] Restoring parameters from /home/mmistroni/tf_logs/rnn-run-20190703193636/model.ckpt-1500
I0703 19:37:19.530741 140195635861376 session_manager.py:500] Running local_init_op.
I0703 19:37:19.536477 140195635861376 session_manager.py:502] Done running local_init_op.
W0703 19:37:19.606058 140195635861376 estimator.py:1000] Input graph does not use tf.data.Dataset or contain a QueueRunner. That means predict yields forever. This is probably a mistake.
I0703 19:37:19.607410 140195635861376 estimator.py:1145] Calling model_fn.


...

Round 7 Prediction: on [[ 0.00331822  0.00755914  0.00251949  0.00145131 -0.00316536  0.00333282
   0.00754994  0.00253552  0.00145788 -0.00317662  0.00333545  0.00754656
   0.00253582  0.00145454 -0.00318083]]=[ 0.00333354  0.00754826  0.00253439  0.00145495 -0.00317876]
<class 'numpy.ndarray'>=[0.003333538770675659, 0.0075482577085494995, 0.0025343894958496094, 0.0014549531042575836, -0.0031787590123713017]
IN Features are:{'rawdata': <tf.Tensor 'concat:0' shape=(1, 15) dtype=float32>}


I0703 19:37:22.754054 140195635861376 estimator.py:1147] Done calling model_fn.
I0703 19:37:22.819294 140195635861376 monitored_session.py:240] Graph was finalized.
I0703 19:37:22.825069 140195635861376 saver.py:1280] Restoring parameters from /home/mmistroni/tf_logs/rnn-run-20190703193636/model.ckpt-1500
I0703 19:37:22.852775 140195635861376 session_manager.py:500] Running local_init_op.
I0703 19:37:22.859030 140195635861376 session_manager.py:502] Done running local_init_op.
W0703 19:37:22.926882 140195635861376 estimator.py:1000] Input graph does not use tf.data.Dataset or contain a QueueRunner. That means predict yields forever. This is probably a mistake.
I0703 19:37:22.928130 140195635861376 estimator.py:1145] Calling model_fn.


Round 8 Prediction: on [[ 0.00333282  0.00754994  0.00253552  0.00145788 -0.00317662  0.00333545
   0.00754656  0.00253582  0.00145454 -0.00318083  0.00333354  0.00754826
   0.00253439  0.00145495 -0.00317876]]=[ 0.00333387  0.00754808  0.00253481  0.00145508 -0.00317907]
<class 'numpy.ndarray'>=[0.003333866596221924, 0.007548078894615173, 0.002534806728363037, 0.0014550834894180298, -0.0031790693756192923]
IN Features are:{'rawdata': <tf.Tensor 'concat:0' shape=(1, 15) dtype=float32>}


I0703 19:37:23.157599 140195635861376 estimator.py:1147] Done calling model_fn.
I0703 19:37:23.224463 140195635861376 monitored_session.py:240] Graph was finalized.
I0703 19:37:23.229152 140195635861376 saver.py:1280] Restoring parameters from /home/mmistroni/tf_logs/rnn-run-20190703193636/model.ckpt-1500
I0703 19:37:23.255382 140195635861376 session_manager.py:500] Running local_init_op.
I0703 19:37:23.260755 140195635861376 session_manager.py:502] Done running local_init_op.


Round 9 Prediction: on [[ 0.00333545  0.00754656  0.00253582  0.00145454 -0.00318083  0.00333354
   0.00754826  0.00253439  0.00145495 -0.00317876  0.00333387  0.00754808
   0.00253481  0.00145508 -0.00317907]]=[ 0.0033339   0.00754803  0.00253481  0.00145499 -0.00317911]
<class 'numpy.ndarray'>=[0.0033338963985443115, 0.007548034191131592, 0.002534806728363037, 0.0014549940824508667, -0.0031791063956916332]


<h3> Now visualizing price predictions against date time </h3>

Installing libraries and downloading data as DataFrame


In [30]:
print('-------------- END OF STORY')
from pprint import pprint
future_days = len(acc) - 15
pprint('Accumlator has length:{}'.format(len(acc)))
pprint('We have prediction for the next:{} days'.format(future_days))
print("calculating business days to zip ")
# we got start date, todays - 15
# end date is  today + 
dts = [START_DATE + BDay(offset) for offset in range(0, len(acc))]
       
print('Start Date:{}={}'.format(dts[0], START_DATE))
print('End date:{}'.format(dts[-1]))
       
zipped = [tpl for tpl in zip(acc, dts)]
       
       
print('===== FULL PREDICTIONS======')
pprint(zipped)


-------------- END OF STORY
'Accumlator has length:65'
'We have prediction for the next:50 days'
calculating business days to zip 
Start Date:2019-06-12 00:00:00=2019-06-12 00:00:00
End date:2019-09-10 00:00:00
===== FULL PREDICTIONS======
[(0.044356058195662795, Timestamp('2019-06-12 00:00:00')),
 (-0.0006350472216118952, Timestamp('2019-06-13 00:00:00')),
 (-0.00480051327272335, Timestamp('2019-06-14 00:00:00')),
 (-0.012272084220461776, Timestamp('2019-06-17 00:00:00')),
 (0.012065445342601011, Timestamp('2019-06-18 00:00:00')),
 (0.007805890702587526, Timestamp('2019-06-19 00:00:00')),
 (-0.012603918399324954, Timestamp('2019-06-20 00:00:00')),
 (-0.007202414159651838, Timestamp('2019-06-21 00:00:00')),
 (0.005171679459776257, Timestamp('2019-06-24 00:00:00')),
 (0.0032870619340650276, Timestamp('2019-06-25 00:00:00')),
 (-0.00783469296684991, Timestamp('2019-06-26 00:00:00')),
 (0.0024407489622544354, Timestamp('2019-06-27 00:00:00')),
 (0.014394117127050965, Timestamp('2019-06-28

In [31]:
from functools import reduce
final_predictions  = acc
#pprint(final_predictions)
# Then find the end price by using reduce and entering the first price 
res = reduce(lambda acc,x: acc * (1+x) ,  final_predictions, first_price)

print('Final Price for series{} = {}'.format(zipped[-1], res))



Final Price for series(-0.0031791063956916332, Timestamp('2019-09-10 00:00:00')) = [159.44086624]
